# Introduction to C/C++

## Logistics

Before we start, make sure you can compile and run a C program.
On Linux or Mac, open a terminal; on Windows, open a Ubuntu Windows
Subsystem for Linux (WSL) terminal.
Then type:
```bash
gcc --version
g++ --version
```
If you see version information, your compiler is ready.
Otherwise, you may need to install your compiler tool chain.
On Linux and WSL, type
```bash
sudo apt update
sudo apt install gcc g++
```
After installation, you should be able to see version information of
`gcc` and `g++`.

## Introduction and Motivation

Modern computational physics often combines two needs:
we want to write code quickly and test ideas fast;
we also want our code to run efficiently on very large problems.
Python is excellent for the first task, but it can be slow when
computations involve many loops or very large arrays.
C and C++ remain the standard tools when speed is critical.
In fact, on the [TIOBE Index](https://www.tiobe.com/tiobe-index), C
and C++ remains 2nd and 3rd places, after python.

![Ken Thompson and Dennis Ritchie](fig/ken+dmr.png)

C was created in the early 1970s by Dennis Ritchie at Bell Labs.
It was used to write the Unix operating system.
Almost every modern language, including Python, is built on top of
ideas from C.
Many scientific codes that you will encounter in research—such as
Gadget for cosmology or AthenaK for astrophysical fluid dynamics—are
written in C or C++.
Knowing C will let you read and even modify these codes.

The main reason C is powerful in computational physics is performance.
C is compiled into machine code, so it runs very fast.
It gives you direct control of memory, which is essential in large
simulations.
With this control comes responsibility: you must allocate and free
memory yourself.
This may seem inconvenient at first, but it teaches you how computers
actually store and move data.

Python and C are not competitors, instead, they complement each other.
You can use Python for prototyping, testing, and visualization.
Once the core of your code is correct, you can rewrite the most
expensive routines in C and call them from Python.
This hybrid approach is common in modern computational physics.

Today's lab will show you how to get started.
We will begin with a very simple numerical method in C/C++.
Then we will step up to a small gravitational simulation.
The goal is not to master every detail of C/C++ in one day, but to
understand why it is useful and to gain the confidence to write simple
programs.

## Essential C Concepts for Python Programmers

If you already know Python, C will feel both familiar and different.
The basic ideas are the same: variables, loops, functions.
But the details are stricter.
C is a compiled, statically typed language.
This means you must tell the computer exactly what kind of data each
variable holds, and the compiler will translate your code into machine
instructions before it runs.

For example, in Python you can write
```python
x = 3
y = 3.5
```
and the interpreter adjusts the type automatically.
In C, you must decide at the start:
```c
int x = 3;
double y = 3.5; 
```
An `int` stores integers, and a `double` stores floating-point
numbers.
You cannot freely switch between them.

Pointers are another new concept.
A pointer is simply the memory address of a variable.
If you declare
```c
double x = 2.0;
double *p = &x;
```
then `p` stores the address of `x`.
You can access the value with `*p`.
Pointers are powerful because they let you share data across functions
without copying.

Functions also look different.
In Python you write
```python
def f(x):
    return x * x
```

In C you must declare the type of the argument and the return value:
```c
double f(double x)
{
    return x * x;
}
```
Every statement ends with a semicolon.
Blocks are enclosed in braces `{ ... }`.

Although C does not really support functional programming,
you can mimic function programming by passing "function pointers".
`f` from the above code is a point to the function `f()`.

C has no built-in lists like Python.
The closest structure is the array.
Arrays have fixed size and do not grow automatically.
For example:
```c
double a[10]; 
```
creates space for 10 floating-point numbers.
You access them by
```c
double x = a[0];
```
where the number in the square bracket is the "offset".

Pointers and array names seem the same thing.
However, they are different.
See [c-faq](https://c-faq.com/aryptr/aryptr2.html) for details.

If you need a flexible array at runtime, you must allocate it
explicitly with malloc.
For instance:
```c
#include <stdlib.h>
...
double *a = (double *)malloc(n * sizeof(double)); 
```
This gives you space for `n` numbers.
When you are done, you must free the memory:
```c
free(a); 
```
This brings us to one of the most important differences.
Python handles memory automatically, while in C you are responsible
for allocating and freeing.
If you forget to free memory, your program will leak memory.
If you access memory after it is freed, your program may crash.
This sounds dangerous, but it also gives you much more control, which
is valuable in performance-critical simulations.

Finally, note that every C program begins execution from a function
called main.
Even if you define many helper functions, the program always starts at
```c
int main(int argc, char *argv[])
{
    ...
    return 0;
}
```
This is the entry point of your program.

With these basic concepts: types, functions, arrays, memory, pointers,
and main, you can already write useful numerical programs.

## Example: Forward Euler

We begin with the forward Euler method.
In Python we wrote a simple function that stepped forward in time by
repeatedly updating the value of `x`.

In [ ]:
# https://ua-2025q3-astr501-513.github.io/notes-8/

import numpy as np

def Euler(f, x, t, dt, n):
    X = [np.array(x)]
    T = [np.array(t)]
    for _ in range(n):
        X.append(X[-1] + dt * f(X[-1]))
        T.append(T[-1] + dt)
    return np.array(X), np.array(T)

def f(x):
    return x

X, T = Euler(f, 1, 0, 0.01, 100)
print(X)
print(T)

Translating the same idea into C requires a bit more work.
```c
/*
 * Paste the following code into "Euler.c".
 * Compile it with `gcc Euler.c -o Euler`.
 * Run with `./Euler` or `./Euler > output.txt`.
 */
#include <stdlib.h>
#include <stdio.h>

struct solution {
	double *X;
	double *T;
};

struct solution
Euler(double (*f)(double), double x, double t, double dt, size_t n)
{
	struct solution s = {
		(double *)malloc((n+1) * sizeof(double)),
		(double *)malloc((n+1) * sizeof(double))
	};

	s.X[0] = x;
	s.T[0] = t;

	for (size_t i = 1; i <= n; ++i) {
		s.X[i] = s.X[i-1] + dt * f(s.X[i-1]);
		s.T[i] = s.T[i-1] + dt;
	}

	return s;
}

void
free_solution(struct solution s)
{
	free(s.X);
	free(s.T);
}

double
f(double x)
{
	return x;
}

int
main(int argc, char *argv[])
{
	size_t n  = 100;
	double dt = 1.0 / n;

	double x0 = 1.0;
	double t0 = 0.0;

	struct solution s = Euler(f, x0, t0, dt, n);

	for (size_t i = 0; i <= n; ++i)
		printf("%.9g ", s.X[i]);
	putchar('\n');
	for (size_t i = 0; i <= n; ++i)
		printf("%.9g ", s.T[i]);
	putchar('\n');

	free_solution(s);

	return 0;
}
```

* Copy and paste the above code into "Euler.c".
* Compile it with `gcc Euler.c -o Euler`.
* Run with `./Euler` or `./Euler > output.txt`.

It create almost identical result as our python code.